In [1]:
from agential.cog.agent.react import ReActAgent
from agential.utils.docstore import DocstoreExplorer

import tiktoken

from langchain_community.docstore.wikipedia import Wikipedia
from langchain_openai import ChatOpenAI

from agential.cog.prompts.benchmark.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.prompts.benchmark.fever import (
    FEVER_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.prompts.benchmark.triviaqa import (
    TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.prompts.benchmark.ambignq import (
    AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.prompts.benchmark.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.prompts.benchmark.svamp import (
    SVAMP_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.prompts.benchmark.tabmwp import (
    TABMWP_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.prompts.benchmark.humaneval import (
    HUMANEVAL_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.prompts.benchmark.mbpp import (
    MBPP_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.prompts.agent.react import (
    REACT_INSTRUCTION_HUMANEVAL, 
    REACT_INSTRUCTION_MBPP, 
    REACT_INSTRUCTION_HOTPOTQA,
    REACT_INSTRUCTION_FEVER,
    REACT_INSTRUCTION_AMBIGNQ,
    REACT_INSTRUCTION_TRIVIAQA,
    REACT_INSTRUCTION_GSM8K,
    REACT_INSTRUCTION_SVAMP,
    REACT_INSTRUCTION_TABMWP
)

import warnings
warnings.filterwarnings('ignore')

import dotenv
dotenv.load_dotenv()

llm = ChatOpenAI()

# QA

### HotpotQA

In [2]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'
question = "What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?"

agent = ReActAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
    max_steps=8,
    max_tokens=3896,
    docstore=DocstoreExplorer(Wikipedia()),
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)

out = agent.generate(
    question=question,
    examples="",#HOTPOTQA_FEWSHOT_EXAMPLES_REACT, 
    prompt=REACT_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reset=True,
    # kwargs.
    max_steps=6
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [3]:
out

[{'thought': 'I need to find out the elevation range for the area that the eastern sector of the Colorado orogeny extends into.',
  'action_type': 'Search',
  'query': 'eastern sector of the Colorado orogeny',
  'observation': 'The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas. This took place from 1780 to 1650 million years ago (Mya), during the Paleoproterozoic (Statherian Period). It is recorded in the Colorado orogen, a >500-km-wide belt of oceanic arc rock that extends southward into New Mexico. The Colorado orogeny was likely part of the larger Yavapai orogeny.'},
 {'thought': 'The passage did not mention the elevation range for the area. I need to search for more information.',
  'action_type': 'Search',
  'query': 'Colorado orogen elevation range',
  'observation': "Could not find [Colorado orogen elevation range]. Similar: ['Appalachian Mountains', 'Geology of North America', 'North American Cordillera', 'Ouachita Mountains

In [ ]:
out_str = ""
for idx, step in enumerate(out):
    out_str += f"Though {idx}: {step['thought']}" + f"Action {idx}: {}"

### FEVER

In [ ]:
question = "Nikolaj Coster-Waldau worked with the Fox Broadcasting Company."

agent = ReActAgent(
    llm=llm,
    mode={"qa": "fever"},
    max_steps=8,
    max_tokens=3896,
    docstore=DocstoreExplorer(Wikipedia()),
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)

out = agent.generate(
    question=question,
    examples=FEVER_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_FEVER,
    additional_keys={},
    reset=True,
    # kwargs.
    max_steps=6
)

In [ ]:
out

### AmbigNQ

In [ ]:
question = "When did the simpsons first air on television?"

agent = ReActAgent(
    llm=llm,
    mode={"qa": "ambignq"},
    max_steps=8,
    max_tokens=3896,
    docstore=DocstoreExplorer(Wikipedia()),
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)

out = agent.generate(
    question=question,
    examples=AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_AMBIGNQ,
    additional_keys={},
    reset=True,
    # kwargs.
    max_steps=6
)

In [ ]:
out

### TriviaQA

In [ ]:
question = "Which American-born Sinclair won the Nobel Prize for Literature in 1930?"

agent = ReActAgent(
    llm=llm,
    mode={"qa": "triviaqa"},
    max_steps=8,
    max_tokens=3896,
    docstore=DocstoreExplorer(Wikipedia()),
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)

out = agent.generate(
    question=question,
    examples=TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_TRIVIAQA,
    additional_keys={},
    reset=True,
    # kwargs.
    max_steps=6
)

In [ ]:
out

# Math

### GSM8K

In [ ]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

agent = ReActAgent(llm=llm, mode={"math": "gsm8k"})

out = agent.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_GSM8K,
)

In [ ]:
out

### SVAMP

In [ ]:
question = "There are 87 oranges and 290 bananas in Philip's collection. If the bananas are organized into 2 groups and oranges are organized into 93 groups. How big is each group of bananas?"

agent = ReActAgent(llm=llm, mode={"math": "svamp"})

out = agent.generate(
    question=question,
    examples=SVAMP_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_SVAMP,
)

In [ ]:
out

### TabMWP

In [ ]:
question = """Read the following table regarding "Bowling Scores" and then write Python code to answer a question:

Name | Score
Amanda | 117
Sam | 236
Irma | 144
Mike | 164

Question: Some friends went bowling and kept track of their scores. How many more points did Mike score than Irma?"""

agent = ReActAgent(llm=llm, mode={"math": "tabmwp"})

out = agent.generate(
    question=question,
    examples=TABMWP_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_TABMWP,
)

In [ ]:
out

# Code

### HumanEval

In [ ]:
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
tests = f"{inst['test']}\ncheck({inst['entry_point']})"

agent = ReActAgent(
    llm=llm,
    mode={"code": "humaneval"},
    max_steps=6,
    max_tokens=3896,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)

out = agent.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_HUMANEVAL,
    additional_keys={},
    reset=True,
    # kwargs.
    max_steps=3
)

In [ ]:
out

### MBPP

In [ ]:
question = "Write a python function to find the first repeated character in a given string."
tests = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""

agent = ReActAgent(
    llm=llm,
    mode={"code": "mbpp"},
    max_steps=6,
    max_tokens=3896,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)

out = agent.generate(
    question=question,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_MBPP,
    additional_keys={"tests": tests},
    reset=True,
    # kwargs.
    max_steps=3
)

In [ ]:
out